# Maximal Marginal Relevance

MMR (Maximal Marginal Relevance) is a powerful diversity-aware retrieval technique used in information retrieval and RAG pipelines to balance relevance and novelty when selecting documents

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [4]:

loader=TextLoader("langchain_rag_dataset.txt")
raw_docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks=splitter.split_documents(raw_docs)
chunks


[Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs).\nLangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='The framework supports integration with various vector databases like FAISS and Chroma for semantic retrieval.\nLangChain enables Retrieval-Augmented Generation (RAG) by allowing developers to fetch relevant context before generating responses.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
 Document(metadata={'

In [5]:
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks, embedding_model)


In [6]:
retriever=vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":3})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x143cfea90>, search_type='mmr', search_kwargs={'k': 3})

In [22]:
prompt=PromptTemplate.from_template("""
                                    Answer the question based on the context provided.

                                    Context:
                                    {context}

                                    Question:
                                    {input}

                                    Answer:""")
llm=init_chat_model("groq:llama-3.1-8b-instant")

In [16]:
document_chain=create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain=create_retrieval_chain(retriever, combine_docs_chain=document_chain)

In [17]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x143cfea90>, search_type='mmr', search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n                                    Answer the question based on the context provided.\n\n                                    Context:\n                                    {context}\n\n                                    Question:\n             

In [21]:
query={"input": "How does langchain support agents and memory?"}
response=rag_chain.invoke(query)

print(f"Answer from LLM:")
print(response)
print("Context:")
for i, context in enumerate(response['context']):
    print(f"\nContext {i+1}: \nPage_content: {context.page_content}")
    print(f"Metadata: {context.metadata}")
print(f"\n\nAnswer: {response['answer']}")


Answer from LLM:
{'input': 'How does langchain support agents and memory?', 'context': [Document(id='2a05a627-5214-4104-be51-d205adbd21f5', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'), Document(id='dbc8a77d-7ea8-430b-b867-8ce2613b40c8', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain agents can interact with external APIs and databases, enhancing the capabilities of LLM-powered applications.\nRAG pipelines in LangChain involve document loading, splitting, embedding, retrieval, and LLM-based response generation.'), Document(id='496373a1-c6ee-43a6-a005-62092464ac7d', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain allows LLMs to act as agents that decide which tool to call and in wha